In [1]:
!nvidia-smi

Tue Feb 15 07:14:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8     2W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model 
from tensorflow.keras.metrics import Precision, AUC, binary_accuracy, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from helper_functions import mcc_metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from transformers import AutoTokenizer

In [3]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [4]:
X = data["SMILES"]

target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MLM', from_pt = True)

In [6]:
smiles_data = tokenizer(list(X),  truncation=True, padding=True)
smiles_data = smiles_data['input_ids']

In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(smiles_data, y_classification, test_size=0.2, random_state=42)
xtrain, xvalid, ytrain, yvalid  = train_test_split(xtrain, ytrain, test_size= 0.16, random_state=42)

In [8]:
len(y_classification)

7807

In [9]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([4956, 2851], dtype=int64))

In [10]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [11]:
class_weights = {0:0.78950695,
                1: 1.36353712} 

In [12]:
word_in_vocab = tokenizer.vocab
len(word_in_vocab)

593

In [13]:
np.array(xtrain[0]).shape

(244,)

In [14]:
input = Input(shape = (244,))

x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=128, 
                        input_length=None,
                        )(input)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(64, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [15]:
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(33), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives()])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)


Epoch 1/60
164/164 [==============================] - ETA: 0s - loss: 0.5427 - binary_accuracy: 0.7548 - auc: 0.7887 - mcc_metric: nan - precision: 0.6867 - true_negatives: 2780.0000 - true_positives: 1179.0000 - false_negatives: 748.0000 - false_positives: 538.0000
Epoch 00001: val_auc improved from -inf to 0.84680, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 50s 247ms/step - loss: 0.5427 - binary_accuracy: 0.7548 - auc: 0.7887 - mcc_metric: nan - precision: 0.6867 - true_negatives: 2780.0000 - true_positives: 1179.0000 - false_negatives: 748.0000 - false_positives: 538.0000 - val_loss: 0.5189 - val_binary_accuracy: 0.7850 - val_auc: 0.8468 - val_mcc_metric: 0.5402 - val_precision: 0.6850 - val_true_negatives: 511.0000 - val_true_positives: 274.0000 - val_false_negatives: 89.0000 - val_false_positives: 126.0000
Epoch 2/60
164/164 [==============================] - ETA: 0s - loss: 0.4985 - binary_accuracy: 0.7935 - auc: 0.8272 - mcc_metric: 0.5576 - precision: 0.7289 - true_negatives: 2818.0000 - true_positives: 1344.0000 - false_negatives: 583.0000 - false_positives: 500.0000
Epoch 00002: val_auc improved from 0.84680 to 0.84814, saving model to F:/bbb/model/nlp\model_33.tf


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


...

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 37s 227ms/step - loss: 0.3056 - binary_accuracy: 0.8660 - auc: 0.9421 - mcc_metric: 0.7174 - precision: 0.7914 - true_negatives: 2880.0000 - true_positives: 1662.0000 - false_negatives: 265.0000 - false_positives: 438.0000 - val_loss: 0.3308 - val_binary_accuracy: 0.8480 - val_auc: 0.9278 - val_mcc_metric: 0.6823 - val_precision: 0.7740 - val_true_negatives: 550.0000 - val_true_positives: 298.0000 - val_false_negatives: 65.0000 - val_false_positives: 87.0000
Epoch 23/60
164/164 [==============================] - ETA: 0s - loss: 0.2966 - binary_accuracy: 0.8633 - auc: 0.9451 - mcc_metric: 0.7159 - precision: 0.7843 - true_negatives: 2859.0000 - true_positives: 1669.0000 - false_negatives: 258.0000 - false_positives: 459.0000
Epoch 00023: val_auc did not improve from 0.92783
164/164 [==============================] - 14s 84ms/step - loss: 0.2966 - binary_accuracy: 0.8633 - auc: 0.9451 - mcc_metric: 0.7159 - precision: 0.7843 - true_negatives: 28

INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


INFO:tensorflow:Assets written to: F:/bbb/model/nlp\model_33.tf\assets


164/164 [==============================] - 38s 235ms/step - loss: 0.2543 - binary_accuracy: 0.8864 - auc: 0.9600 - mcc_metric: 0.7628 - precision: 0.8204 - true_negatives: 2945.0000 - true_positives: 1704.0000 - false_negatives: 223.0000 - false_positives: 373.0000 - val_loss: 0.3581 - val_binary_accuracy: 0.8430 - val_auc: 0.9359 - val_mcc_metric: 0.6884 - val_precision: 0.7309 - val_true_negatives: 517.0000 - val_true_positives: 326.0000 - val_false_negatives: 37.0000 - val_false_positives: 120.0000
Epoch 28/60
164/164 [==============================] - ETA: 0s - loss: 0.2616 - binary_accuracy: 0.8803 - auc: 0.9578 - mcc_metric: 0.7516 - precision: 0.8051 - true_negatives: 2903.0000 - true_positives: 1714.0000 - false_negatives: 213.0000 - false_positives: 415.0000
Epoch 00028: val_auc did not improve from 0.93589
164/164 [==============================] - 14s 85ms/step - loss: 0.2616 - binary_accuracy: 0.8803 - auc: 0.9578 - mcc_metric: 0.7516 - precision: 0.8051 - true_negatives: 2

In [16]:
model.load_weights(save(33))
model.evaluate(np.array(xtest), np.array(ytest))

49/49 [==============================] - 2s 39ms/step - loss: 0.4269 - binary_accuracy: 0.8201 - auc: 0.9147 - mcc_metric: 0.6361 - precision: 0.7035 - true_negatives: 797.0000 - true_positives: 484.0000 - false_negatives: 77.0000 - false_positives: 204.0000


[0.4268687963485718,
 0.8201024532318115,
 0.9147198796272278,
 0.6360712051391602,
 0.7034883499145508,
 797.0,
 484.0,
 77.0,
 204.0]

In [17]:
model.evaluate(np.array(xvalid), np.array(yvalid))

32/32 [==============================] - 1s 39ms/step - loss: 0.3581 - binary_accuracy: 0.8430 - auc: 0.9359 - mcc_metric: 0.6884 - precision: 0.7309 - true_negatives: 517.0000 - true_positives: 326.0000 - false_negatives: 37.0000 - false_positives: 120.0000


[0.35811635851860046,
 0.8429999947547913,
 0.9358888268470764,
 0.6884199976921082,
 0.7309417128562927,
 517.0,
 326.0,
 37.0,
 120.0]

In [18]:
tf.__version__

'2.7.0'